In [3]:
import sys
import os
kernel_name = os.path.basename(sys.executable.replace("/bin/python",""))
kernel_name

'torch_env2'

In [ ]:
# TODO: I am not conviced that I am extracting the image tiles in the correct way.

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as keras_preprocess_input

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
import torch
import os
import re
import scanpy as sc
import cv2
import numpy as np
from PIL import Image
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import umap

In [ ]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
#model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.fc = torch.nn.Identity()  # remove last layer

In [ ]:
current_folder = globals()['_dh'][0]
visium_dir = current_folder / ".." / ".." / "data" / "uscsc_dump"
spot_images = current_folder / ".." / ".." / "data" / "spot_images"

In [ ]:
samples = [f for f in os.listdir(visium_dir) if f.startswith("visium")]
{i: smp for i, smp in enumerate(samples)}

In [ ]:
smp = samples[3]
print(smp)
base_name = re.sub(r"\.h5ad$", "", smp)
base_name = re.sub(r"^visium_", "", base_name)
vis_adata = sc.read_h5ad(visium_dir / smp)
vis_adata.var_names_make_unique()
print("Number of spots:", vis_adata.shape[0])

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as keras_preprocess_input

In [ ]:
# TODO: I am not sure whether I am reading in the images in the correct way
# See here: https://pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html
transform = ResNet50_Weights.IMAGENET1K_V2.transforms()
features = np.zeros((len(vis_adata), 2048))
#for i, img_name in enumerate(imgs):
for i, barcode in enumerate(vis_adata.obs_names):
    img_path = str(spot_images / base_name / (barcode + ".jpeg"))

    # see here: https://github.com/BiomedicalMachineLearning/stLearn/blob/09c8d7a79979268fe78273fcb25726c5e94c2c6c/stlearn/image_preprocessing/feature_extractor.py#L60
    # and here: https://github.com/BiomedicalMachineLearning/stLearn/blob/09c8d7a79979268fe78273fcb25726c5e94c2c6c/stlearn/image_preprocessing/model_zoo.py#LL54C12-L54C12
    # we need to make sure to use the same transforms as in the original training
    img = Image.open(img_path)
    img = transform(img)
    img = img.unsqueeze(0)
    features[i, :] = model.forward(img).detach().numpy()

In [ ]:
# make pca of the features
pca_coords = PCA(n_components=50).fit_transform(features)

In [ ]:
# plot the first two components, color by vis_adata.obs.leiden using seaborn
sns.scatterplot(x=pca_coords[:, 0], y=pca_coords[:, 1], hue=vis_adata.obs.leiden)
plt.show()

In [ ]:
vis_adata.obs.columns

In [ ]:
# make alpha = 0.2
sns.scatterplot(x=pca_coords[:, 0], y=pca_coords[:, 1], hue=vis_adata.obs.p53, alpha=0.4)
plt.show()

In [ ]:
# make umap of the features
umap_coords = umap.UMAP(n_components=2).fit_transform(features)

In [ ]:
# plot umap coords, color by vis_adata.obs.leiden using seaborn
sns.scatterplot(x=umap_coords[:, 0], y=umap_coords[:, 1], hue=vis_adata.obs.leiden, alpha=0.4)

# Appendix

In [ ]:
barcode = vis_adata.obs_names[0]
img_path = str(spot_images / base_name / (barcode + ".jpeg"))
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img)
img = img.resize((224, 224))
img = np.array(img)
img = np.transpose(img, (2, 0, 1))
img = img / 255
img = torch.from_numpy(img).float()
img = img.unsqueeze(0)
img.shape

In [ ]:
barcode = vis_adata.obs_names[0]
img_path = str(spot_images / base_name / (barcode + ".jpeg"))
img = Image.open(img_path)
img = np.asarray(img, dtype="int32")
img = img.astype(np.float32)
img = np.stack([img])

# The images are converted from RGB to BGR, then each color channel is zero-centered with respect to the ImageNet dataset, without scaling.

img = np.transpose(img, (0, 3, 1, 2))
img = torch.from_numpy(img).float()
model.forward(img).detach().numpy()

In [ ]:
barcode = vis_adata.obs_names[10]
img_path = str(spot_images / base_name / (barcode + ".jpeg"))
img = Image.open(img_path)
img